In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow_probability as tfp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

import sys
# TODO: Change root path
root = "/Users/jay/Desktop/Bachelorarbeit"

sys.path.append(f"{root}/Implementation")
from src.execute_model import run_model_single_parameter_node
from src.construct_model import get_model


ndims = 7
dims = ["TT", "C0", "beta", "ETF", "FC", "FRAC", "K2"]
colors = sns.color_palette(n_colors=ndims)

vizConfigPath = f"{root}/viz_config.json"
with open(vizConfigPath, "r") as file:
    viz_config = json.load(file)

configPath = viz_config["configPath"]
basis = viz_config["basis"]
input_file = viz_config["input_file"]
sep_viz = viz_config["sep_viz"] if "sep_viz" in viz_config else False
monte_carlo_repetition = viz_config["monte_carlo_repetition"] if "monte_carlo_repetition" in viz_config else 1000
model = get_model(configPath, basis)

start_date: 2004-01-01 00:00:00
start_date_predictions: 2005-01-01 00:00:00
end_date: 2006-01-01 00:00:00
simulation length: 365
full_data_range is 732 hours including spin_up_length of 366 hours
simulation_range is of length 366 hours


In [2]:
# Construct params
configurationObject = model.configurationObject
param_lower = []
param_upper = []
for param in configurationObject["parameters"]:
    if param["distribution"] == "Uniform":
        param_lower.append(param["lower"])
        param_upper.append(param["upper"])
    else:
        raise NotImplementedError(
            f"Sorry, the distribution {param['distribution']} is not supported yet"
        )
param_lower = np.array(param_lower)
param_upper = np.array(param_upper)

In [3]:
samples = pd.read_csv(f"{root}/{input_file}")
samples

,TT_1,C0_1,beta_1,ETF_1,FC_1,FRAC_1,K2_1,TT_2,C0_2,beta_2,...,FC_7,FRAC_7,K2_7,TT_8,C0_8,beta_8,ETF_8,FC_8,FRAC_8,K2_8
0,2.596915,2.398848,2.306367,0.011929,206.840242,0.209709,0.049248,2.921993,2.767528,1.737159,...,271.300171,0.238786,0.038774,2.357434,1.706845,2.575770,0.011253,231.258552,0.293089,0.046370
1,2.596915,2.398848,2.306367,0.011929,206.840242,0.209709,0.049248,2.921993,2.767528,1.737159,...,271.300171,0.238786,0.038774,2.357434,1.706845,2.575770,0.011253,231.258552,0.293089,0.046370
2,2.652492,2.448110,1.926934,0.023314,224.045539,0.221674,0.044358,2.921993,2.767528,1.737159,...,328.603552,0.226606,0.038645,2.357434,1.706845,2.575770,0.011253,231.258552,0.293089,0.046370
3,2.652492,2.448110,1.926934,0.023314,224.045539,0.221674,0.044358,2.581229,2.216106,1.895625,...,328.603552,0.226606,0.038645,2.910239,2.553436,2.495525,0.007014,229.833625,0.281555,0.045549
4,2.652492,2.448110,1.926934,0.023314,224.045539,0.221674,0.044358,2.581229,2.216106,1.895625,...,328.603552,0.226606,0.038645,2.910239,2.553436,2.495525,0.007014,229.833625,0.281555,0.045549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.628170,2.235095,1.838511,0.003079,224.471299,0.209689,0.049778,2.613238,2.225591,1.828833,...,229.661528,0.225765,0.049814,2.628270,2.240789,1.872118,0.018588,226.454243,0.215349,0.049648
996,2.628170,2.235095,1.838511,0.003079,224.471299,0.209689,0.049778,2.613238,2.225591,1.828833,...,229.661528,0.225765,0.049814,2.628270,2.240789,1.872118,0.018588,226.454243,0.215349,0.049648
997,2.628170,2.235095,1.838511,0.003079,224.471299,0.209689,0.049778,2.613238,2.225591,1.828833,...,234.628452,0.229401,0.049999,2.628270,2.240789,1.872118,0.018588,226.454243,0.215349,0.049648
998,2.628170,2.235095,1.838511,0.003079,224.471299,0.209689,0.049778,2.613238,2.225591,1.828833,...,234.628452,0.229401,0.049999,2.628270,2.240789,1.872118,0.018588,226.454243,0.215349,0.049648


# Functions

In [4]:
def merge_samples(samples):
    df = pd.DataFrame()
    for key in ['TT', 'C0', 'beta', 'ETF', 'FC', 'FRAC', 'K2']:
        selected_columns = samples.filter(regex=f'^{key}_').values
        df[key] = selected_columns.flatten()
    return df
        

def sep_record_check(samples):
    return True if len(samples.columns) > 7 else False


def histogram(samples):
    # Merge samples
    if sep_record_check(samples):
        samples = merge_samples(samples)

    fig = make_subplots(rows=4, cols=2, specs=[[{}, {}], [{}, {}], [{}, {}], [{}, None]])

    # Adding a histogram and KDE 
    for i, col in enumerate(samples.columns):
        row = (i // 2) + 1 
        col_idx = (i % 2) + 1 

        # Histogram
        fig.add_trace(
            go.Histogram(x=samples[col], name=col, histnorm="probability density"),
            row=row,
            col=col_idx,
        )

        # KDE calculation
        kde = gaussian_kde(samples[col])
        x_values = np.linspace(samples[col].min(), samples[col].max(), 300)
        kde_values = kde(x_values)

        # Adding KDE
        fig.add_trace(
            go.Scatter(
                x=x_values, y=kde_values, mode="lines", name=f"KDE {col}", showlegend=False
            ),
            row=row,
            col=col_idx,
        )

    # Update layout
    fig.update_layout(
        height=1800,
        width=1200,
        title_text="Markov chain Monte Carlo: Parameters Overview",
    )
    fig.show()


def boxplot(samples):
    # Merge samples
    if sep_record_check(samples):
        samples = merge_samples(samples)

    fig = make_subplots(rows=4, cols=2, specs=[[{}, {}], [{}, {}], [{}, {}], [{}, None]])

    # Adding boxplot
    for i, col in enumerate(samples.columns):
        row = (i // 2) + 1 
        col_idx = (i % 2) + 1

        fig.add_trace(
            go.Box(
                y=samples[col],
                name=col,
                boxpoints="all",
                jitter=0.5,
                whiskerwidth=0.2,
                marker_size=2,
                line_width=1,
            ),
            row=row,
            col=col_idx,
        )

    # Update layout 
    fig.update_layout(
        height=1800,
        width=1200,
        title_text="Markov chain Monte Carlo: Boxplots of Parameters",
    )
    fig.show()


def heatmap(samples):
    # Merge samples
    if sep_record_check(samples):
        samples = merge_samples(samples)

    corr_matrix = samples.corr()

    # Create the heatmap
    fig = go.Figure(
        data=go.Heatmap(
            z=corr_matrix,
            x=corr_matrix.columns,
            y=corr_matrix.columns,
            colorscale="Viridis", 
            colorbar=dict(title="Correlation Coefficient"),
        )
    )

    # Update layout
    fig.update_layout(
        title="Markov chain Monte Carlo: Heatmap of Variable Correlations",
        xaxis_title="Variables",
        yaxis_title="Variables",
    )

    fig.show()


def sep_heatmap(samples):
    nchains = len(samples.columns) // 7
    for i in range(nchains):
        start_index = 7 * i
        end_index = 7 * (i + 1)
        df = samples.iloc[:, start_index:end_index]
        heatmap(df)


def sep_histogram(samples):
    histogram_colors = [
        "#636EFA",
        "#EF553B",
        "#00CC96",
        "#AB63FA",
        "#FFA15A",
        "#19D3F3",
        "#FF6692",
        "#B6E880",
        "#FF97FF",
        "#FECB52",
    ]
    kde_colors = [
        "#FFD700",
        "#00BFFF",
        "#FF6347",
        "#FFFF33",
        "#4682B4",
        "#FF4500",
        "#32CD32",
        "#800080",
        "#008000",
        "#00008B",
    ]

    chains_data = []
    nchains = len(samples.columns) // 7

    # Load data
    for i in range(nchains):
        start_index = 7 * i
        end_index = 7 * (i + 1)
        df = samples.iloc[:, start_index:end_index]
        df.columns = ['TT', 'C0', 'beta', 'ETF', 'FC', 'FRAC', 'K2']
        chains_data.append(df)
    combined_df = merge_samples(samples)
    chains_data.append(combined_df)

    parameters = ['TT', 'C0', 'beta', 'ETF', 'FC', 'FRAC', 'K2']
    num_params = 7

    # Create subplots
    fig = make_subplots(
        rows=num_params,
        cols=nchains + 1,
        subplot_titles=[
            (
                f"{parameters[param]} (Combined)"
                if chain == nchains
                else f"{parameters[param]} (Chain {chain + 1})"
            )
            for param in range(num_params)
            for chain in range(nchains + 1)
        ],
    )

    # Populate subplots
    for chain_idx, df in enumerate(chains_data):
        for param_idx, param in enumerate(df.columns):
            data = df[param]

            # Histogram
            fig.add_trace(
                go.Histogram(
                    x=data,
                    name=f"{param} Chain {chain_idx+1}",
                    histnorm="probability density",
                    opacity=0.75,
                    marker_color=histogram_colors[param_idx],
                ),
                row=param_idx + 1,
                col=chain_idx + 1,
            )

            # KDE calculation
            kde = gaussian_kde(data)
            x_values = np.linspace(data.min(), data.max(), 300)
            kde_values = kde(x_values)

            # Adding KDE
            fig.add_trace(
                go.Scatter(
                    x=x_values,
                    y=kde_values,
                    mode="lines",
                    name=f"KDE {param}",
                    showlegend=False,
                    line=dict(color=kde_colors[param_idx]),
                ),
                row=param_idx + 1,
                col=chain_idx + 1,
            )

    # Update layout
    fig.update_layout(
        height=2000,
        width=2400,
        showlegend=False,
        title_text=f"Markov chain Monte Carlo: Parameters Overview by Chain",
    )
    fig.show()


def sep_boxplot(samples):
    histogram_colors = [
        "#636EFA",
        "#EF553B",
        "#00CC96",
        "#AB63FA",
        "#FFA15A",
        "#19D3F3",
        "#FF6692",
        "#B6E880",
        "#FF97FF",
        "#FECB52",
    ]

    chains_data = []
    nchains = len(samples.columns) // 7

    # Load data
    for i in range(nchains):
        start_index = 7 * i
        end_index = 7 * (i + 1)
        df = samples.iloc[:, start_index:end_index]
        df.columns = ['TT', 'C0', 'beta', 'ETF', 'FC', 'FRAC', 'K2']
        chains_data.append(df)
    combined_df = merge_samples(samples)
    chains_data.append(combined_df)

    parameters = ['TT', 'C0', 'beta', 'ETF', 'FC', 'FRAC', 'K2']
    num_params = 7

    # Create subplots
    fig = make_subplots(
        rows=num_params,
        cols=nchains + 1,
        subplot_titles=[
            (
                f"{parameters[param]} (Combined)"
                if chain == nchains
                else f"{parameters[param]} (Chain {chain + 1})"
            )
            for param in range(num_params)
            for chain in range(nchains + 1)
        ],
    )

    # Populate subplots
    for chain_idx, df in enumerate(chains_data):
        for param_idx, param in enumerate(df.columns):
            data = df[param]

            # Boxplot
            fig.add_trace(
                go.Box(
                    y=data,
                    name=f"{param} Chain {chain_idx+1}",
                    marker_color=histogram_colors[param_idx],
                    showlegend=False,
                ),
                row=param_idx + 1,
                col=chain_idx + 1,
            )

    # Update layout
    fig.update_layout(
        height=2000,
        width=2400,
        showlegend=False,
        title_text=f"Markov chain Monte Carlo: Boxplot Overview by Chain",
    )
    fig.show()


def sep_heatmap(samples):
    num_params = 7
    nchains = len(samples.columns) // 7

    fig = make_subplots(
        rows=1, cols=nchains + 1,
        subplot_titles=['Combined' if i == nchains else f'Chain {i + 1}' for i in range(nchains + 1)],
        shared_yaxes=True
    )

    # Generate a heatmap for each chain
    for i in range(nchains):
        start_index = num_params * i
        end_index = num_params * (i + 1)
        df = samples.iloc[:, start_index:end_index]
        df.columns = ['TT', 'C0', 'beta', 'ETF', 'FC', 'FRAC', 'K2']

        corr_matrix = df.corr()

        heatmap = go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.columns,
            colorscale="Viridis",
            colorbar=dict(title="Correlation Coefficient"),
            name=f'Chain {i + 1}'
        )

        fig.add_trace(heatmap, row=1, col=i + 1)

    df = merge_samples(samples)
    corr_matrix = df.corr()

    heatmap = go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale="Viridis",
        colorbar=dict(title="Correlation Coefficient"),
        name=f'Combined'
    )

    fig.add_trace(heatmap, row=1, col=nchains + 1)
    

    # Update layout
    fig.update_layout(
        title="Comparison of Variable Correlations Across Chains",
        height=600,
        width=300 * nchains
    )

    fig.show()



# Plotting

In [5]:
sep_histogram(samples) if sep_viz else histogram(samples)

In [6]:
sep_boxplot(samples) if sep_viz else boxplot(samples)

In [7]:
sep_heatmap(samples) if sep_viz else heatmap(samples)

# Sampling

In [8]:
# Merge samples
if sep_record_check(samples):
    samples = merge_samples(samples)
    

# Sampling Max
param_vec = []
for i in range(len(samples.loc[0])):
    values, counts = np.unique(samples.iloc[:, i], return_counts=True)
    ind = np.argmax(counts)
    param_vec.append(values[ind])

_, posterior_max, _, _ = run_model_single_parameter_node(model, param_vec)


# Mean Sampling from MCMC
sample_param = []
for i in range(7):
    sample_param.append(np.random.choice(samples.iloc[:, i], monte_carlo_repetition))
sample_param = np.array(sample_param).T

posterior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    posterior.append(y_model)

print(np.array(posterior).shape)
posterior_mean = np.mean(np.array(posterior), axis=0)


# Mean Sampling from Prior
sample_param = []
for i in range(7):
    uni = tfp.distributions.Uniform(low=param_lower[i], high=param_upper[i]).sample(1)
    sample_param.append(uni)
sample_param = np.array(sample_param).T

prior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    prior.append(y_model)

_, _, measured_data, _ = run_model_single_parameter_node(model, np.array(vec))

prior_means = np.mean(np.array(prior), axis=0)

[HVBSASK INFO] [0] parameters: [[2.659607517410457, 2.355747271411151, 1.924049549712317, 0.013894989046512946, 225.9051681636026, 0.21074808782527213, 0.049957980022905066]]
[HVBSASK INFO] [0] parameters: [array([2.87147459e+00, 2.63030290e+00, 1.78119731e+00, 1.38949890e-02,
       2.27319384e+02, 2.10748088e-01, 4.81183395e-02])]
[HVBSASK INFO] [0] parameters: [array([2.62801094e+00, 2.28379602e+00, 1.71685344e+00, 1.99473866e-02,
       2.40537967e+02, 2.05866840e-01, 4.93713527e-02])]
[HVBSASK INFO] [0] parameters: [array([3.13067303e+00, 2.21606290e+00, 1.89753221e+00, 4.83794568e-02,
       2.28255945e+02, 2.11420490e-01, 4.99135001e-02])]
[HVBSASK INFO] [0] parameters: [array([3.13067303e+00, 2.35017979e+00, 1.92404955e+00, 6.20130463e-02,
       2.27319384e+02, 2.76324812e-01, 4.98820050e-02])]
[HVBSASK INFO] [0] parameters: [array([2.62347775e+00, 2.13148438e+00, 2.01974308e+00, 6.66542975e-02,
       2.25905168e+02, 2.00819949e-01, 4.99688484e-02])]
[HVBSASK INFO] [0] parame

2024-06-19 21:51:30.376087: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-19 21:51:30.376111: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-06-19 21:51:30.376116: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-06-19 21:51:30.376135: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-19 21:51:30.376148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
# Create a figure
fig = go.Figure()
model.get_start_date
dates = model.get_simulation_range()
fig.add_trace(
    go.Scatter(
        x=dates,
        y=prior_means,
        mode="lines",
        name="Prior Mean",
        line=dict(color="lightgrey"),
    )
)
fig.add_trace(
    go.Scatter(x=dates, y=posterior_mean, mode="lines", name="Posterior Mean")
)
fig.add_trace(go.Scatter(x=dates, y=posterior_max, mode="lines", name="Posterior Max"))
fig.add_trace(go.Scatter(x=dates, y=measured_data, mode="lines", name="Measured Data"))

# Update layout
fig.update_layout(
    title="Markov chain Monte Carlo: Bayesian Inference Result Comparison",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Time Series",
    hovermode="x unified",
    template="plotly_white",
)

# Show the figure
fig.show()

In [10]:
def rmse(result, target):
    diff = result - target
    aggr = 0
    for i in range(len(diff)):
        aggr += diff[i] ** 2
    rmse = (aggr / (len(diff))) ** 0.5
    return rmse


def mae(result, target):
    return np.absolute(result - target).mean()


print(f"RMSE of Posterior Mean: {rmse(posterior_mean, measured_data)}")
print(f"RMSE of Posterior Max: {rmse(posterior_max, measured_data)}")
print(f"MAE of Posterior Mean: {mae(posterior_mean, measured_data)}")
print(f"MAE of Posterior Max: {mae(posterior_max, measured_data)}")

RMSE of Posterior Mean: 20.377175016099986
RMSE of Posterior Max: 20.860841982963898
MAE of Posterior Mean: 9.289698890449273
MAE of Posterior Max: 9.43161561755508
